### Pipeline Raw to Silver

In [1]:
import pandas as pd 
import numpy as np
from pathlib import Path
import psycopg2
from psycopg2 import extras

#### Definições e Configurações

In [2]:
# Definição de caminhos 
DATA_PATH: Path = Path("../Data Layer/silver/transactions_cards_users_mcc_silver.csv")
DDL_PATH: Path = Path("../Transformer/scripts/transactions_cards_users_mcc_silver_ddl.sql")

# Parâmetros de conexão do Postgres
conn_params: dict = {
  "host": "localhost",
  "port": 5433,
  "dbname": "transactions",
  "user": "admin",
  "password": "admin"
}

#### Funções auxiliares

In [3]:
def convert_sql_to_string(file: Path) -> str:
  """
  Recebe: Caminho para arquivo SQL (DDL);
  Retorna: String com a instrução DDL para criação da tabela.
  """
  try:
    with open(file, 'r') as f:
      return f.read()
  except Exception as e:
    print(f"ERRO! Falha ao encontrar/ler o arquivo: {e}")
    raise

#### Funções de ETL

In [4]:
def extract(path: Path) -> pd.DataFrame:
  """
  Recebe: Caminho para arquivo CSV.
  Retorna: DataFrame;
  """
  try:
    print(f"Extraindo dados de {path}...")
    df = pd.read_csv(path)
    print(f"Extração foi concluída - Linhas extraídas: {len(df)}.")
    return df
  except Exception as e:
    print(f"ERRO! Um problema ocorreu na conversão do arquivo para DataFrame: {e}")
    return None
  
def transform_and_validate(df: pd.DataFrame) -> pd.DataFrame:
  """
  Recebe: DataFrame e realiza limpeza e conversão de tipos.
  Retorna: DataFrame validado e compatível com psycopg2.
  """
  try:
    print("Iniciando a transformação, padronização e tipagem dos dados...")
    
    # Dados em Datas
    date_cols = ['date', 'expires', 'acct_open_date']
    for col in date_cols:
      df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    # Dados Numéricos
    int_cols = ['mcc', 'num_cards_issued', 'current_age', 'retirement_age', 'credit_score', 'num_credit_cards', 'zip', 'cvv', 'year_pin_last_changed']
    float_cols = ['amount', 'credit_limit', 'per_capita_income', 'yearly_income', 'total_debt']

    for col in int_cols:
      df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int32')
    for col in float_cols:
      df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)

    string_cols = ['merchant_id', 'card_number', 'use_chip', 'merchant_city', 'merchant_state', 'errors', 'card_brand', 'card_type', 'has_chip', 'gender', 'address', 'mcc_description'
    ]

    # Dados em Strings
    for col in string_cols:
      if col in df.columns:
          df[col] = (
              df[col]
              .astype(str)
              .replace({
                  "<NA>": None,
                  "nan": None,
                  "NaN": None,
                  "None": None,
                  "": None,
                  " ": None
              }, regex=False).str.strip()
          )

    if 'use_chip' in df.columns:
      df['use_chip'] = df['use_chip'].replace({
        'chip inserted': 'Chip',
        'swiped': 'Swipe',
        'manual': 'Manual'
      })

    if 'merchant_state' in df.columns:
      df['merchant_state'] = df['merchant_state'].str.upper()

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df.astype(object).where(pd.notnull(df), None)

    print("Transformação e padronização concluídas com sucesso.")
    return df

  except Exception as e:
    print(f"ERRO! O DataFrame não foi validado: {e}")
    return None

def load(df: pd.DataFrame, ddl_script_path: Path, conn_params: dict) -> bool:
    try:
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()

        print(f"Lendo o arquivo DDL: {ddl_script_path.name}")
        ddl = ddl_script_path.read_text(encoding="utf-8")
        print(f"Criando a tabela {ddl_script_path.stem}...")
        cur.execute(ddl)
        conn.commit()
        print("Tabela criada no Banco de Dados!")

        table = ddl_script_path.stem.replace("_silver_ddl", "")
        columns = list(df.columns)
        cols = ', '.join(columns)
        insert_query = f"INSERT INTO silver.{table} ({cols}) VALUES ({', '.join(['%s'] * len(columns))})"

        print(f"Inserindo {len(df)} linhas...")
        tuples = [tuple(x) for x in df.itertuples(index=False, name=None)]
        extras.execute_batch(cur, insert_query, tuples, page_size=20000)
        conn.commit()

        print("LOAD no PostgreSQL concluído com sucesso.")
        return True

    except Exception as e:
        print(f"ERRO: Processo de load interrompido: {e}")
        if conn:
            conn.rollback()
        return False

    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

#### Pipeline

In [5]:
def run_pipeline(data_path: Path, ddl_script_path: Path, conn_params: dict) -> bool:
  """
  Executa o Pipeline 
  """
  try:
    print("ETAPA 01: Extração de dados")
    print("Executando...")
    df_raw = extract(data_path)
    if df_raw is None: 
      print("FALHA NO PIPELINE: Extração.")
      return False
    print(f"Extração concluida, DataFrame carregado. Linhas: {len(df_raw)}")


    print("\nETAPA 02: Transformação e Validação dos dados")
    print("Executando...")
    df_silver = transform_and_validate(
      df_raw, 
    )
    if df_silver is None: 
      print("FALHA NO PIPELINE: Transformação e Validação.")
      return False 
    
    print("\nETAPA 03: Carregamento dos dados (LOAD)")
    success = load(df_silver, ddl_script_path, conn_params)
    if not success:
      print("FALHA NO PIPELINE: Carregamento (LOAD)")

    print("\nPIPELINE CONCLUÍDA COM SUCESSO!")
    return True
  except Exception as e: 
    print(f"ERRO GERAL: {e}")
    return False

In [6]:
run_pipeline(
    DATA_PATH, 
    DDL_PATH, 
    conn_params
)

ETAPA 01: Extração de dados
Executando...
Extraindo dados de ../Data Layer/silver/transactions_cards_users_mcc_silver.csv...
Extração foi concluída - Linhas extraídas: 2045955.
Extração concluida, DataFrame carregado. Linhas: 2045955

ETAPA 02: Transformação e Validação dos dados
Executando...
Iniciando a transformação, padronização e tipagem dos dados...
Transformação e padronização concluídas com sucesso.

ETAPA 03: Carregamento dos dados (LOAD)
Lendo o arquivo DDL: transactions_cards_users_mcc_silver_ddl.sql
Criando a tabela transactions_cards_users_mcc_silver_ddl...
Tabela criada no Banco de Dados!
Inserindo 2045955 linhas...
LOAD no PostgreSQL concluído com sucesso.

PIPELINE CONCLUÍDA COM SUCESSO!


True